In [4]:
import requests
from datetime import datetime, timedelta

VALD_CONFIG = {
    "client_id": "HDgfl7yGMdC1s6w==",
    "client_secret": "21patT6fiHd5KAkwCaiGytIXv62mVODc2eg=",
    "tenant_id": "28887524-c9e0-4783-84a8-5d6ecc41c239",
    "token_url": "https://security.valdperformance.com/connect/token",
    "dynamo_base_url": "https://prd-use-api-extdynamo.valdperformance.com"
}

def get_access_token():
    token_data = {
        "grant_type": "client_credentials",
        "client_id": VALD_CONFIG["client_id"],
        "client_secret": VALD_CONFIG["client_secret"]
    }
    response = requests.post(VALD_CONFIG["token_url"], data=token_data)
    if response.ok:
        return response.json()["access_token"]
    return None

def fetch_dynamo_tests():
    token = get_access_token()
    if not token:
        print("Failed to get token")
        return
    
    headers = {"Authorization": f"Bearer {token}"}
    
    # Date parameters - all three are REQUIRED
    now = datetime.now()
    test_from = (now - timedelta(days=90)).strftime('%Y-%m-%dT00:00:00.000Z')  # 90 days ago
    test_to = now.strftime('%Y-%m-%dT23:59:59.000Z')  # Now
    modified_from = (now - timedelta(days=90)).strftime('%Y-%m-%dT00:00:00.000Z')  # 90 days ago
    
    tenant_id = VALD_CONFIG['tenant_id']
    base = VALD_CONFIG['dynamo_base_url']
    
    # Correct endpoint structure per documentation
    url = f"{base}/v2022q2/teams/{tenant_id}/tests"
    
    params = {
        "modifiedFromUtc": modified_from,
        "testFromUtc": test_from,
        "testToUtc": test_to,
        "includeRepSummaries": "true"  # Include the aggregated data
    }
    
    print(f"URL: {url}")
    print(f"Params: {params}\n")
    
    response = requests.get(url, headers=headers, params=params, timeout=15)
    
    print(f"Status: {response.status_code}")
    
    if response.ok:
        data = response.json()
        print(f"Response type: {type(data)}")
        
        if isinstance(data, dict):
            print(f"Keys: {data.keys()}")
            items = data.get("items", data.get("tests", []))
        elif isinstance(data, list):
            items = data
        else:
            items = []
            
        print(f"Found {len(items)} tests")
        
        if items:
            print("\n" + "="*50)
            print("FIRST TEST:")
            print("="*50)
            first = items[0]
            for key, value in first.items():
                # Truncate long values
                val_str = str(value)
                if len(val_str) > 100:
                    val_str = val_str[:100] + "..."
                print(f"  {key}: {val_str}")
    else:
        print(f"Error: {response.text}")

fetch_dynamo_tests()

URL: https://prd-use-api-extdynamo.valdperformance.com/v2022q2/teams/28887524-c9e0-4783-84a8-5d6ecc41c239/tests
Params: {'modifiedFromUtc': '2025-09-24T00:00:00.000Z', 'testFromUtc': '2025-09-24T00:00:00.000Z', 'testToUtc': '2025-12-23T23:59:59.000Z', 'includeRepSummaries': 'true'}

Status: 200
Response type: <class 'dict'>
Keys: dict_keys(['items', 'currentPage', 'totalItems', 'totalPages'])
Found 50 tests

FIRST TEST:
  id: 8390a59f-beb1-44c1-83bb-00ab5e1b64c2
  athleteId: 99c7d903-9544-407f-abe9-167ddcf1c227
  teamId: 28887524-c9e0-4783-84a8-5d6ecc41c239
  testCategory: Strength
  bodyRegion: Shoulder
  movement: Abduction
  position: SeatedLongLever
  customPosition: 
  laterality: RightSide
  attachments: LeftPalmPadRightCurvedPad
  leftAttachment: None
  rightAttachment: None
  startTimeUTC: 2025-10-01T20:18:48.807065Z
  durationSeconds: 29.783071
  hardwareInfo: DynaMoLite-01357;DYNL-221221-FW0R2-HW0R5
  softwareInfo: iOS;iPhone17,1;18.6.2;1.9.9;#114
  analysisInfo: Dynamo.Analy

In [5]:
import requests
from datetime import datetime, timedelta
import pandas as pd

VALD_CONFIG = {
    "client_id": "HDgfl7yGMdC1s6w==",
    "client_secret": "21patT6fiHd5KAkwCaiGytIXv62mVODc2eg=",
    "tenant_id": "28887524-c9e0-4783-84a8-5d6ecc41c239",
    "token_url": "https://security.valdperformance.com/connect/token",
    "dynamo_base_url": "https://prd-use-api-extdynamo.valdperformance.com"
}

def get_access_token():
    token_data = {
        "grant_type": "client_credentials",
        "client_id": VALD_CONFIG["client_id"],
        "client_secret": VALD_CONFIG["client_secret"]
    }
    response = requests.post(VALD_CONFIG["token_url"], data=token_data)
    if response.ok:
        return response.json()["access_token"]
    return None

def fetch_all_dynamo_tests():
    """Fetch all Dynamo tests with pagination"""
    token = get_access_token()
    if not token:
        print("Failed to get token")
        return []
    
    headers = {"Authorization": f"Bearer {token}"}
    
    now = datetime.now()
    test_from = (now - timedelta(days=90)).strftime('%Y-%m-%dT00:00:00.000Z')
    test_to = now.strftime('%Y-%m-%dT23:59:59.000Z')
    modified_from = (now - timedelta(days=90)).strftime('%Y-%m-%dT00:00:00.000Z')
    
    tenant_id = VALD_CONFIG['tenant_id']
    base = VALD_CONFIG['dynamo_base_url']
    url = f"{base}/v2022q2/teams/{tenant_id}/tests"
    
    all_tests = []
    page = 1
    
    while True:
        params = {
            "modifiedFromUtc": modified_from,
            "testFromUtc": test_from,
            "testToUtc": test_to,
            "includeRepSummaries": "true",
            "page": page
        }
        
        response = requests.get(url, headers=headers, params=params, timeout=15)
        
        if response.ok:
            data = response.json()
            items = data.get("items", [])
            all_tests.extend(items)
            
            total_pages = data.get("totalPages", 1)
            print(f"Page {page}/{total_pages} - Retrieved {len(items)} tests")
            
            if page >= total_pages:
                break
            page += 1
        else:
            print(f"Error on page {page}: {response.text}")
            break
    
    return all_tests

# Fetch all tests
all_tests = fetch_all_dynamo_tests()
print(f"\nTotal tests retrieved: {len(all_tests)}")

# Analyze the data
print("\n" + "="*60)
print("TEST CATEGORIES:")
print("="*60)
categories = {}
for test in all_tests:
    cat = test.get('testCategory', 'Unknown')
    categories[cat] = categories.get(cat, 0) + 1
for cat, count in sorted(categories.items(), key=lambda x: -x[1]):
    print(f"  {cat}: {count}")

print("\n" + "="*60)
print("BODY REGIONS:")
print("="*60)
regions = {}
for test in all_tests:
    region = test.get('bodyRegion', 'Unknown')
    regions[region] = regions.get(region, 0) + 1
for region, count in sorted(regions.items(), key=lambda x: -x[1]):
    print(f"  {region}: {count}")

print("\n" + "="*60)
print("MOVEMENTS:")
print("="*60)
movements = {}
for test in all_tests:
    movement = test.get('movement', 'Unknown')
    movements[movement] = movements.get(movement, 0) + 1
for movement, count in sorted(movements.items(), key=lambda x: -x[1]):
    print(f"  {movement}: {count}")

print("\n" + "="*60)
print("TEST TYPES (bodyRegion + movement + position):")
print("="*60)
test_types = {}
for test in all_tests:
    test_type = f"{test.get('bodyRegion', '')} {test.get('movement', '')} - {test.get('position', '')}"
    test_types[test_type] = test_types.get(test_type, 0) + 1
for tt, count in sorted(test_types.items(), key=lambda x: -x[1]):
    print(f"  {tt}: {count}")

print("\n" + "="*60)
print("SAMPLE repetitionTypeSummaries STRUCTURE:")
print("="*60)
# Find a test with rep summaries
for test in all_tests:
    reps = test.get('repetitionTypeSummaries', [])
    if reps:
        print(f"Test: {test.get('bodyRegion')} {test.get('movement')} - {test.get('position')}")
        print(f"Laterality: {test.get('laterality')}")
        print(f"Number of rep summaries: {len(reps)}")
        print("\nFirst rep summary:")
        for key, value in reps[0].items():
            print(f"  {key}: {value}")
        break

print("\n" + "="*60)
print("UNIQUE ATHLETES:")
print("="*60)
athletes = set(test.get('athleteId') for test in all_tests)
print(f"  {len(athletes)} unique athletes")

Page 1/36 - Retrieved 50 tests
Page 2/36 - Retrieved 50 tests
Page 3/36 - Retrieved 50 tests
Page 4/36 - Retrieved 50 tests
Page 5/36 - Retrieved 50 tests
Page 6/36 - Retrieved 50 tests
Page 7/36 - Retrieved 50 tests
Page 8/36 - Retrieved 50 tests
Page 9/36 - Retrieved 50 tests
Page 10/36 - Retrieved 50 tests
Page 11/36 - Retrieved 50 tests
Page 12/36 - Retrieved 50 tests
Page 13/36 - Retrieved 50 tests
Page 14/36 - Retrieved 50 tests
Page 15/36 - Retrieved 50 tests
Page 16/36 - Retrieved 50 tests
Page 17/36 - Retrieved 50 tests
Page 18/36 - Retrieved 50 tests
Page 19/36 - Retrieved 50 tests
Page 20/36 - Retrieved 50 tests
Page 21/36 - Retrieved 50 tests
Page 22/36 - Retrieved 50 tests
Page 23/36 - Retrieved 50 tests
Page 24/36 - Retrieved 50 tests
Page 25/36 - Retrieved 50 tests
Page 26/36 - Retrieved 50 tests
Page 27/36 - Retrieved 50 tests
Page 28/36 - Retrieved 50 tests
Page 29/36 - Retrieved 50 tests
Page 30/36 - Retrieved 50 tests
Page 31/36 - Retrieved 50 tests
Page 32/36 - Retr